In [1]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('./data/reviews_Video_Games_5.json.gz')

In [ ]:
df.shape

In [2]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1.0,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4.0,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1.0,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]","I got this version instead of the PS3 version,...",3.0,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,DIRT 3,1308009600,"06 14, 2011"


In [3]:
rev_text = df['summary']
ratings = df['overall']

In [5]:
import re

review_new = []
for text in rev_text:
    string = re.sub("[-\s+\.\!\/_,$%^*(+\"\']+|[+——！，。?、~@#￥%……&*()（） ]+", " ", text)
    review_new.append(string)
#print(lists_new)

In [6]:
# 去掉首尾空格
review = []
for i in review_new:
    i = i.strip()
    review.append(i)

In [8]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split


import argparse
import sys

import numpy as np
import pandas
from sklearn import metrics
import tensorflow as tf

learn = tf.contrib.learn


train_data, test_data, train_target, test_target = train_test_split(review, ratings, random_state=1234)

In [56]:
global n_words
# 处理词汇
MAX_DOCUMENT_LENGTH = 50
MIN_WORD_FREQUENCE = 1
EMBEDDING_SIZE = 50

vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH, min_frequency=MIN_WORD_FREQUENCE)
x_train = np.array(list(vocab_processor.fit_transform(train_data)))
x_test = np.array(list(vocab_processor.transform(test_data)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Total words: 17955


In [57]:
def rnn_model(features, target):
    
    # Convert indexes of words into embeddings.
    # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
    # maps word indexes of the sequence into [batch_size, sequence_length,
    # EMBEDDING_SIZE].
    word_vectors = tf.contrib.layers.embed_sequence(
            features, vocab_size=n_words, embed_dim=EMBEDDING_SIZE, scope='words')

    # Split into list of embedding per word, while removing doc length dim.
    # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].
    word_list = tf.unstack(word_vectors, axis=1)

    # Create a Gated Recurrent Unit cell with hidden size of EMBEDDING_SIZE.
    cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

    # Create an unrolled Recurrent Neural Networks to length of
    # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
    _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

    target = tf.one_hot(target, 15, 1, 0)
    logits = tf.contrib.layers.fully_connected(encoding, 15, activation_fn= None)
    loss = tf.contrib.losses.softmax_cross_entropy(logits, target)

    # Create a training op.
    train_op = tf.contrib.layers.optimize_loss(
            loss,
            tf.contrib.framework.get_global_step(),
            optimizer='Adam',
            learning_rate=1e-4)

    return ({
            'class': tf.argmax(logits, 1),
            'prob': tf.nn.softmax(logits)
    }, loss, train_op)

In [58]:
y_train = pandas.Series(train_target)
y_test = pandas.Series(test_target)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [59]:
model_fn = rnn_model
classifier = learn.SKCompat(learn.Estimator(model_fn=model_fn))

# Train and predict
classifier.fit(x_train, y_train, steps=10000)
y_predicted = classifier.predict(x_test)['class']
score = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy: {0:f}'.format(score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020A8F949408>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\Dovelism\\AppData\\Local\\Temp\\tmpp0u8gfvj'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done r

INFO:tensorflow:global_step/sec: 20.6753
INFO:tensorflow:loss = 0.98288167, step = 7101 (4.835 sec)
INFO:tensorflow:global_step/sec: 19.5414
INFO:tensorflow:loss = 1.0255967, step = 7201 (5.119 sec)
INFO:tensorflow:global_step/sec: 18.9363
INFO:tensorflow:loss = 0.86626345, step = 7301 (5.280 sec)
INFO:tensorflow:global_step/sec: 18.6458
INFO:tensorflow:loss = 1.0038377, step = 7401 (5.364 sec)
INFO:tensorflow:global_step/sec: 18.8878
INFO:tensorflow:loss = 1.0673053, step = 7501 (5.294 sec)
INFO:tensorflow:global_step/sec: 19.0587
INFO:tensorflow:loss = 1.0881844, step = 7601 (5.247 sec)
INFO:tensorflow:global_step/sec: 18.1539
INFO:tensorflow:loss = 0.98111886, step = 7701 (5.512 sec)
INFO:tensorflow:global_step/sec: 18.3481
INFO:tensorflow:loss = 0.89933133, step = 7801 (5.447 sec)
INFO:tensorflow:global_step/sec: 16.9228
INFO:tensorflow:loss = 1.0080802, step = 7901 (5.910 sec)
INFO:tensorflow:global_step/sec: 19.5065
INFO:tensorflow:loss = 0.9877783, step = 8001 (5.126 sec)
INFO:t